In [ ]:
import pandas as pd

In [ ]:
def add_ones(X):
    return np.hstack((np.ones((X.shape[0], 1)), X))


def least_squares(csv):
    """Parses a urls pair string into urls pair."""
    #print("CEEYESVEE",csv,csv.shape)
    #nums = csv.split(",")
    #num1 = float(nums[0])
    #num2 = float(nums[1])

    #X = np.asmatrix(np.array(num1))
    #T = np.asmatrix(np.array(num2))
    #X = add_ones(X)
    T = np.asmatrix(csv)
    T = np.hstack((np.ones((T.shape[0], 1)), T))
    xtx = T.T.dot(T)

    #arr = ["weights",np.array([1,1*num2,num2*1,num2*num2])]
    arr = ["weights",xtx]

    return arr

def interval_calc(csv1,csv2):
    #nums = csv.split(",")
    #num1 = float(nums[0])
    #num2 = float(nums[1])

    #X = np.asmatrix(np.array(num1))
    #T = np.asmatrix(np.array(num2))
    X = csv1
    T = csv2
    print("X",X)
    print("T",T)
    X = np.asmatrix(X)
    T = np.asmatrix(T)
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    interval = X.T.dot(T)
    arr = ["weights",interval]
    return arr

In [ ]:
import numpy as np 
from abc import ABC, abstractmethod

# Super class for machine learning models 

class BaseModel(ABC):
    """ Super class for ITCS Machine Learning Class"""
    
    @abstractmethod
    def train(self, X, T):
        pass

    @abstractmethod
    def use(self, X):
        pass

    
class LinearModel(BaseModel):
    """
        Abstract class for a linear model 
        
        Attributes
        ==========
        w       ndarray
                weight vector/matrix
    """

    def __init__(self):
        """
            weight vector w is initialized as None
        """
        self.w = None

    # check if the matrix is 2-dimensional. if not, raise an exception    
    def _check_matrix(self, mat, name):
        if len(mat.shape) != 2:
            raise ValueError(''.join(["Wrong matrix ", name]))
        
    # add a basis
    def add_ones(self, X):
        """
            add a column basis to X input matrix
        """
        self._check_matrix(X, 'X')
        return np.hstack((np.ones((X.shape[0], 1)), X))

    ####################################################
    #### abstract funcitons ############################
    @abstractmethod
    def train(self, X, T):
        """
            train linear model
            
            parameters
            -----------
            X     2d array
                  input data
            T     2d array
                  target labels
        """        
        pass
    
    @abstractmethod
    def use(self, X):
        """
            apply the learned model to input X
            
            parameters
            ----------
            X     2d array
                  input data
            
        """        
        pass 

In [ ]:
# Linear Regression Class for least squares
class LinearRegress(LinearModel): 
    """ 
        LinearRegress class 
        
        attributes
        ===========
        w    nd.array  (column vector/matrix)
             weights
    """
    def __init__(self):
        LinearModel.__init__(self)
        
    # train lease-squares model
    def train(self, X, T):
        X = self.add_ones(X)
        print("xshape",X.shape)
        print("Tshape",T.shape)
        xtx = X.T.dot(X)
        #print(xtx)
        print("xtx shape",xtx.shape)
        self.w = np.linalg.pinv(xtx)
        print(X.T.shape)
        print(T.shape)
        interval = (X.T).dot(T)
        print("interval shape",interval.shape)
        self.w = self.w.dot(interval)
        print("w shape",self.w.shape)
        self.w = self.w.T
        print(self.w)
        return self.w.T
        ## TODO: replace this with your codes
    """
    def train_parallel(self, X, T):
        X = self.add_ones(X)
        print("xshape",X.shape)
        print("Tshape",T.shape)
        
        loop_through = spark.sparkContext.parallelize([x for x in X])
        links = loop_through.map(lambda word: least_squares(word)).reduceByKey(lambda a,b:a+b)
        for link,rank in links.collect():
            print(link,rank)
            xtx = rank
        
        loop_through = spark.sparkContext.parallelize([ (x,y) for x,y in zip(X,T)])
        links2 = loop_through.map(lambda word: interval_calc(word[1],word[0])).reduceByKey(lambda a,b:a+b)

        for link,rank in links2.collect():
            interval = rank

        xtx = np.linalg.pinv(xtx)

        for link,rank in links2.collect():
            interval = rank

        rank = np.asmatrix(rank)
        rank = rank


        print("FINAL WEIGHTS = ")
        weights = xtx.dot(rank)
        self.w = weights
        return self.w
    """
    # apply the learned model to data X
    def use(self, X):
        X = self.add_ones(X)
        hypothesis = self.w.dot(X.T)
        return hypothesis.T
        ## TODO: replace this with your codes

In [ ]:
import collections # for checking iterable instance

# LMS class 
class LMS(LinearModel):
    """
        Lease Mean Squares. online learning algorithm
    
        attributes
        ==========
        w        nd.array
                 weight matrix
        alpha    float
                 learning rate
    """
    def __init__(self, alpha):
        LinearModel.__init__(self)
        self.alpha = alpha
        
    # batch training by using train_step function
    def train(self, X, T):
        iterations = 3000
        X = self.add_ones(X)
        self.w = np.zeros((1,X.shape[1]))
        #print("X is ")
        #print(X)
        #print("W is")
        #print(self.w)
        for i in range(iterations):            
            temp = self.w.dot(X.T)
            error = temp.T - T
            new_x = error.T.dot(X)
            self.w = self.w - ((self.alpha)*new_x)
        print("train response")
        return self.w
        pass  ## TODO: replace this with your codes
            
    # train LMS model one step 
    # here the x is 1d vector
    def train_step(self,x,t):
        x = np.insert(x,0,1)
        x = x.reshape(1,len(x))
        #print(x)
        self.w = np.zeros((1,x.shape[1]))
        #print(self.w)
        #self.w = np.zeros((1,x.shape[1]))
        mult_x = x
        error = self.w.dot(mult_x.T) - t
        #print(error)
                                                 
        update_value = ((self.alpha)*error)*x
        self.w = self.w - update_value
        print(self.w)
        return self.w
        
        
    def train_step_full(self, x, t):
        x = self.add_ones(x)
        self.w = np.zeros((1,x.shape[1]))
        print(x[0])
        for i in range(x.shape[1]):
            #print(self.w)
            mult_x = np.reshape(x[i],(len(x[i]),1))
            error = self.w.dot(mult_x) - t[i]
            update_value = ((self.alpha)*error)*x[i]
            self.w = self.w - update_value
            #print(self.w)
        print("returning stochastic")
        return self.w
        
        pass  ## TODO: replace this with your codes
    
    # apply the current model to data X
    def use(self, X):
        X = self.add_ones(X)
        hypothesis = self.w.dot(X.T)
        return hypothesis.T
        pass  ## TODO: replace this with your codes
        

In [6]:
randf = pd.read_csv("song_ratings",sep=",")
randf
X =randf.iloc[:,:2].values
T = randf.iloc[:,2].values


X = np.asmatrix(X)

T = np.asmatrix(T)
T = T.T
T.shape
linreg = LinearRegress()
print(X.shape)
#linreg = LMS(0.002)
linreg.train(X,T)
useout = linreg.use(X)
print(useout)

(3, 2)
xshape (3, 3)
Tshape (3, 1)
xtx shape (3, 3)
(3, 3)
(3, 1)
interval shape (3, 1)
w shape (3, 1)
[[ 2.66057515  7.01452047 -4.35394532]]
[[9.56141096]
 [3.99086273]
 [9.44772631]]


In [7]:
randf1 = pd.read_csv("echonest.csv",sep=",")

In [8]:
print("done")

done


In [9]:
randf1 = randf1.iloc[:100000,:]

In [10]:
randf1.columns.values

array(['track_id', 'echonest', 'echonest.1', 'echonest.2', 'echonest.3',
       'echonest.4', 'echonest.5', 'echonest.6', 'echonest.7',
       'echonest.8', 'echonest.9', 'echonest.10', 'echonest.11',
       'echonest.12', 'echonest.13', 'echonest.14', 'echonest.15',
       'echonest.16', 'echonest.17', 'echonest.18', 'echonest.19',
       'echonest.20', 'echonest.21', 'echonest.22', 'echonest.23',
       'echonest.24', 'echonest.25', 'echonest.26', 'echonest.27',
       'echonest.28', 'echonest.29', 'echonest.30', 'echonest.31',
       'echonest.32', 'echonest.33', 'echonest.34', 'echonest.35',
       'echonest.36', 'echonest.37', 'echonest.38', 'echonest.39',
       'echonest.40', 'echonest.41', 'echonest.42', 'echonest.43',
       'echonest.44', 'echonest.45', 'echonest.46', 'echonest.47',
       'echonest.48', 'echonest.49', 'echonest.50', 'echonest.51',
       'echonest.52', 'echonest.53', 'echonest.54', 'echonest.55',
       'echonest.56', 'echonest.57', 'echonest.58', 'echonest.

In [11]:
randf1 = randf1.iloc[:,:8]

In [12]:
randf1

,track_id,echonest,echonest.1,echonest.2,echonest.3,echonest.4,echonest.5,echonest.6
0,2,0.416675,0.675894,0.634476,1.062807e-02,0.177647,0.159310,165.922
1,3,0.374408,0.528643,0.817461,1.851103e-03,0.105880,0.461818,126.957
2,5,0.043567,0.745566,0.701470,6.967990e-04,0.373143,0.124595,100.260
3,10,0.951670,0.658179,0.924525,9.654270e-01,0.115474,0.032985,111.562
4,134,0.452217,0.513238,0.560410,1.944269e-02,0.096567,0.525519,114.290
5,139,0.106550,0.260911,0.607067,8.350870e-01,0.223676,0.030569,196.961
6,140,0.376312,0.734079,0.265685,6.695811e-01,0.085995,0.039068,107.952
7,141,0.963657,0.435933,0.075632,3.454935e-01,0.105686,0.026658,33.477
8,142,0.662881,0.379065,0.823856,9.102662e-01,0.088705,0.079090,147.781
9,144,0.909011,0.443643,0.641997,9.240917e-01,0.267669,0.089659,128.537


In [13]:
df = randf1

In [14]:
for item in list(df.columns.values):
    na_len = df[item].isna().sum()
    total_len = df[item].count()
    
    if na_len > 0.7*total_len:
        del df[item]

In [15]:
df

,track_id,echonest,echonest.1,echonest.2,echonest.3,echonest.4,echonest.5,echonest.6
0,2,0.416675,0.675894,0.634476,1.062807e-02,0.177647,0.159310,165.922
1,3,0.374408,0.528643,0.817461,1.851103e-03,0.105880,0.461818,126.957
2,5,0.043567,0.745566,0.701470,6.967990e-04,0.373143,0.124595,100.260
3,10,0.951670,0.658179,0.924525,9.654270e-01,0.115474,0.032985,111.562
4,134,0.452217,0.513238,0.560410,1.944269e-02,0.096567,0.525519,114.290
5,139,0.106550,0.260911,0.607067,8.350870e-01,0.223676,0.030569,196.961
6,140,0.376312,0.734079,0.265685,6.695811e-01,0.085995,0.039068,107.952
7,141,0.963657,0.435933,0.075632,3.454935e-01,0.105686,0.026658,33.477
8,142,0.662881,0.379065,0.823856,9.102662e-01,0.088705,0.079090,147.781
9,144,0.909011,0.443643,0.641997,9.240917e-01,0.267669,0.089659,128.537


In [16]:
columns = list(df.columns.values)
col = 0

df.loc[df.index.dropna()]

for index,row in df.iterrows():
    removed = False
    col = 0
    while col < len(columns) and removed == False:
        if row[columns[col]] != row[columns[col]]:
            df.drop(index, inplace=True)
            removed = True
        col = col+1

In [18]:
def cleanup_data(df):
    for item in list(df.columns.values):
        na_len = df[item].isna().sum()
        total_len = df[item].count()

        if na_len > 0.7*total_len:
            del df[item]
    columns = list(df.columns.values)
    col = 0

    df.loc[df.index.dropna()]

    for index,row in df.iterrows():
        removed = False
        col = 0
        while col < len(columns) and removed == False:
            if row[columns[col]] != row[columns[col]]:
                df.drop(index, inplace=True)
                removed = True
            col = col+1

In [19]:
df

,track_id,echonest,echonest.1,echonest.2,echonest.3,echonest.4,echonest.5,echonest.6
0,2,0.416675,0.675894,0.634476,1.062807e-02,0.177647,0.159310,165.922
1,3,0.374408,0.528643,0.817461,1.851103e-03,0.105880,0.461818,126.957
2,5,0.043567,0.745566,0.701470,6.967990e-04,0.373143,0.124595,100.260
3,10,0.951670,0.658179,0.924525,9.654270e-01,0.115474,0.032985,111.562
4,134,0.452217,0.513238,0.560410,1.944269e-02,0.096567,0.525519,114.290
5,139,0.106550,0.260911,0.607067,8.350870e-01,0.223676,0.030569,196.961
6,140,0.376312,0.734079,0.265685,6.695811e-01,0.085995,0.039068,107.952
7,141,0.963657,0.435933,0.075632,3.454935e-01,0.105686,0.026658,33.477
8,142,0.662881,0.379065,0.823856,9.102662e-01,0.088705,0.079090,147.781
9,144,0.909011,0.443643,0.641997,9.240917e-01,0.267669,0.089659,128.537


In [20]:
raw_tracks_df = pd.read_csv("raw_tracks.csv",sep=",")

In [21]:
raw_tracks_df.columns

Index(['track_id', 'album_id', 'album_title', 'album_url', 'artist_id',
       'artist_name', 'artist_url', 'artist_website', 'license_image_file',
       'license_image_file_large', 'license_parent_id', 'license_title',
       'license_url', 'tags', 'track_bit_rate', 'track_comments',
       'track_composer', 'track_copyright_c', 'track_copyright_p',
       'track_date_created', 'track_date_recorded', 'track_disc_number',
       'track_duration', 'track_explicit', 'track_explicit_notes',
       'track_favorites', 'track_file', 'track_genres', 'track_image_file',
       'track_information', 'track_instrumental', 'track_interest',
       'track_language_code', 'track_listens', 'track_lyricist',
       'track_number', 'track_publisher', 'track_title', 'track_url'],
      dtype='object')

In [22]:
raw_tracks_df = raw_tracks_df[["track_id","album_id","album_title","artist_id","artist_name","track_interest","track_title","track_genres"]]

In [23]:
raw_tracks_df

,track_id,album_id,album_title,artist_id,artist_name,track_interest,track_title,track_genres
0,2,1.0,AWOL - A Way Of Life,1,AWOL,4656,Food,"[{'genre_id': '21', 'genre_title': 'Hip-Hop', ..."
1,3,1.0,AWOL - A Way Of Life,1,AWOL,1470,Electric Ave,"[{'genre_id': '21', 'genre_title': 'Hip-Hop', ..."
2,5,1.0,AWOL - A Way Of Life,1,AWOL,1933,This World,"[{'genre_id': '21', 'genre_title': 'Hip-Hop', ..."
3,10,6.0,Constant Hitmaker,6,Kurt Vile,54881,Freeway,"[{'genre_id': '10', 'genre_title': 'Pop', 'gen..."
4,20,4.0,Niris,4,Nicky Cook,978,Spiritual Level,"[{'genre_id': '76', 'genre_title': 'Experiment..."
5,26,4.0,Niris,4,Nicky Cook,1060,Where is your Love?,"[{'genre_id': '76', 'genre_title': 'Experiment..."
6,30,4.0,Niris,4,Nicky Cook,718,Too Happy,"[{'genre_id': '76', 'genre_title': 'Experiment..."
7,46,4.0,Niris,4,Nicky Cook,252,Yosemite,"[{'genre_id': '76', 'genre_title': 'Experiment..."
8,48,4.0,Niris,4,Nicky Cook,247,Light of Light,"[{'genre_id': '76', 'genre_title': 'Experiment..."
9,134,1.0,AWOL - A Way Of Life,1,AWOL,1126,Street Music,"[{'genre_id': '21', 'genre_title': 'Hip-Hop', ..."


In [24]:
raw_tracks_df.shape

(109727, 8)

In [25]:
cleanup_data(raw_tracks_df)

In [26]:
raw_tracks_df.iloc[0]["track_genres"]

"[{'genre_id': '21', 'genre_title': 'Hip-Hop', 'genre_url': 'http://freemusicarchive.org/genre/Hip-Hop/'}]"

In [27]:
print("done")

done


In [43]:
df.to_csv("feature_short.csv", sep=',', encoding='utf-8')

In [44]:
print("done")

done


In [29]:
feature_data = df.values

In [30]:
track_data = raw_tracks_df.values

In [31]:
np.hstack((feature_data[feature_data[:,0]==2][:,1:],np.asmatrix(2)))

matrix([[4.16675233e-01, 6.75893985e-01, 6.34476268e-01, 1.06280683e-02,
         1.77646571e-01, 1.59310065e-01, 1.65922000e+02, 2.00000000e+00]])

In [39]:
def understand_user(username):
    df = pd.read_csv(username + ".csv",sep=",",header=None).values
    #vertical_data = []
    X = feature_data[feature_data[:,0]==df[0][0]][:,1:]
    for item in df[1:]:
        print(item[0])
        print(item[0].shape)
        print(item[1])
        print(feature_data[feature_data[:,0]==item[0]][:,1:].shape)
        temp_data =  feature_data[feature_data[:,0]==item[0]][:,1:]
        
        print(temp_data.shape)
        X = np.vstack((X,temp_data))
        
    
    print(X)
    T = np.asmatrix(df[:,1]).T
    print(T)
    linreg = LinearRegress()
    print(X.shape)
    #linreg = LMS(0.002)
    linreg.train(X,T)
    print("linregw",linreg.w)
    return linreg
    

In [40]:
linreg = understand_user("anirudh")

3
()
7
(1, 7)
(1, 7)
5
()
8
(1, 7)
(1, 7)
[[4.16675233e-01 6.75893985e-01 6.34476268e-01 1.06280683e-02
  1.77646571e-01 1.59310065e-01 1.65922000e+02]
 [3.74407769e-01 5.28643062e-01 8.17461132e-01 1.85110320e-03
  1.05879944e-01 4.61818128e-01 1.26957000e+02]
 [4.35668989e-02 7.45565870e-01 7.01469992e-01 6.96799000e-04
  3.73143312e-01 1.24595342e-01 1.00260000e+02]]
[[10]
 [ 7]
 [ 8]]
(3, 7)
xshape (3, 8)
Tshape (3, 1)
xtx shape (8, 8)
(8, 3)
(3, 1)
interval shape (8, 1)
w shape (8, 1)
[[ 1.05906299 -1.78046548  2.19032109 -0.1270727   0.00438049  1.99369322
  -2.13098623  0.04983234]]
linregw [[ 1.05906299 -1.78046548  2.19032109 -0.1270727   0.00438049  1.99369322
  -2.13098623  0.04983234]]


In [34]:
#picking random and using model on

def recommend(linreg):
    X = feature_data[100:200,0:]
    X2 = track_data[100:200,0:][:,track_data.shape[1]-2:track_data.shape[1]-1]
    useout = linreg.use(X[:,1:])
    useout[useout>10]=10
    print(np.asmatrix(X[:,0]).T.shape)
    track_rec = np.hstack((X2,np.asmatrix(X[:,0]).T,useout))
    track_rec = np.array(track_rec)
    track_rec = track_rec[track_rec[:,2].argsort()][::-1]
    return track_rec

In [37]:
val = recommend(linreg)

(100, 1)


In [38]:
val

array([['Roach', 579.0, 10.0],
       ['vermalderhide', 549.0, 10.0],
       ['Red coated young girlfriend', 539.0, 10.0],
       ['Clapartroach', 563.0, 9.943647517691574],
       ['Oh Kap Soon', 574.0, 9.92070969337681],
       ['Wonder Inn', 580.0, 9.611516030913991],
       ['The Swamp', 544.0, 9.45860198683855],
       ["My Lil'", 342.0, 9.455031624915216],
       ['Julie Andrews', 532.0, 9.328657389684736],
       ['The Natives are Restless', 398.0, 9.242882913102667],
       ['The Hotel', 366.0, 8.906474016104806],
       ['Hippopotamus', 529.0, 8.835953543738865],
       ['My Room', 537.0, 8.660301842987424],
       ['The Tides Of Land', 569.0, 8.624296843981696],
       ['Where are all the people', 367.0, 8.582185394702186],
       ['Excretia', 526.0, 8.500559112049313],
       ['Disco Ghoul', 524.0, 8.477008832714557],
       ['Untitled Boogie (edit), Transfiguration Boogie, Boogie Hole',
        565.0, 8.398955139530788],
       ['Lucky 45', 562.0, 8.390803563574796],
      